<a href="https://colab.research.google.com/github/HromovAndrey/II_Homework_mood_6_part_3/blob/main/%D0%94%D0%97_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* Завантажте дані
* Розділіть дані на тренувальні та тестові
* У `MyTokenizer` добавте обробку тексту за допомогою `nltk`, застосуйте **стематизацію**
* Застосуйте `TfidfVectorizer`
* Застосуйте `TruncatedSVD`
* Натренуйте модель [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) з параметром `kernel="linear"`, якщо вистачить пам'яті можна спробувати `kernel="rbf"`
* Виведіть результати

# Дані про телеграм

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module6/data/telegram_spam.csv")

In [4]:
df.head()

,text_type,text
0,spam,naturally irresistible your corporate identity...
1,spam,the stock trading gunslinger fanny is merrill ...
2,spam,unbelievable new homes made easy im wanting to...
3,spam,4 color printing special request additional in...
4,spam,do not have money get software cds from here s...


In [6]:
X = df['text_type']
y = df['text']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
class MyTokenizer:
    def __init__(self):
        self.stemmer = PorterStemmer()

    def __call__(self, text):
        tokens = word_tokenize(text.lower())  # Токенизация и преобразование в нижний регистр
        stemmed_tokens = [self.stemmer.stem(token) for token in tokens]  # Стемматизация
        return stemmed_tokens

In [10]:
vectorizer = TfidfVectorizer(tokenizer=MyTokenizer(), stop_words='english')

In [11]:
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', '

In [12]:
pca = TruncatedSVD(n_components=min(300, X_train_tfidf.shape[1]))

In [13]:
X_train_reduced = pca.fit_transform(X_train_tfidf)
X_test_reduced = pca.transform(X_test_tfidf)

X_train_reduced.shape, X_test_reduced.shape

((16278, 2), (4070, 2))

In [ ]:
svc_linear = SVC(kernel="linear", class_weight="balanced")
svc_linear.fit(X_train_reduced, y_train)

In [ ]:
y_pred_linear = svc_linear.predict(X_test_reduced)

In [ ]:
print("Результаты для SVC с kernel='linear':")
print(classification_report(y_test, y_pred_linear))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_linear, display_labels=svc_linear.classes_)
plt.xticks(rotation=90)
plt.title("Confusion Matrix for SVC with kernel='linear'")
plt.show()

In [ ]:
try:
    svc_rbf = SVC(kernel="rbf", class_weight="balanced")
    svc_rbf.fit(X_train_reduced, y_train)

    # Предсказание и оценка модели SVC с kernel="rbf"
    y_pred_rbf = svc_rbf.predict(X_test_reduced)

    # Выводим отчет о классификации
    print("Результаты для SVC с kernel='rbf':")
    print(classification_report(y_test, y_pred_rbf))

    # Построение матрицы ошибок
    ConfusionMatrixDisplay.from_predictions(y_test, y_pred_rbf, display_labels=svc_rbf.classes_)
    plt.xticks(rotation=90)
    plt.title("Confusion Matrix for SVC with kernel='rbf'")
    plt.show()

except MemoryError:
    print("Недостаточно памяти для выполнения модели SVC с kernel='rbf'.")